# Preprocesamiento de datasets

## Configuración inicial y librerías

In [1]:
import pandas as pd # Librería para la manipulación y el análisis de datos
import numpy as np # Librería para la manipulación de datos y para la ejecución de operaciones matemáticas
import matplotlib.pyplot as plt # Librería para la visualización de datos
import seaborn as sns # Librería para la visualización de datos
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder # Librería para crear modelos de ML
import os
# conectar con drive
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/data/TFM/'

Mounted at /content/drive


## DATASET 'CUSTUMER_COMERCIAL_ACTIVITY.CSV'

In [2]:
cca = pd.read_csv(os.path.join(data_path, 'customer_commercial_activity.csv'),index_col=0)
cca.sample(3)

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
6811327,274932,2018-10,2015-02,KFC,1.0,02 - PARTICULARES
9030636,1427208,2019-01,2018-08,KHQ,0.0,03 - UNIVERSITARIO
3390179,1178238,2018-06,2016-09,KAT,1.0,02 - PARTICULARES


In [3]:
cca['entry_date'].describe()

,entry_date
count,5962924
unique,53
top,2016-10
freq,382473


#### Tratamos cada una de las 6 columnas:

In [4]:
# pk_cid: Identificador de cliente

cca['pk_cid'].nunique()
cca['pk_cid'].describe()
cca['pk_cid'].isnull().sum()

np.int64(0)

In [5]:
# pk_partition: Identificador de partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes

# Cambiamos a datetime
cca['pk_partition'] = pd.to_datetime(cca['pk_partition'])
cca['pk_partition'].isnull().sum() # 0

np.int64(0)

In [6]:
# entry_date: Fecha en la que se realizó la primera contratación a través de EasyMoney

# Cambiamos a datetime
cca['entry_date'] = pd.to_datetime(cca['entry_date'])
cca['entry_date'].isnull().sum() # 0


np.int64(0)

In [7]:
# Crear un campo booleano para saber si el cliente se ha unido hace menos de un año

last_date = cca['entry_date'].max()
cca['new_customer'] = (last_date - cca['entry_date']).dt.days < 365
cca['new_customer'] = cca['new_customer'].astype(int)

In [8]:
cca.new_customer.value_counts()

,count
new_customer,
0,4963581
1,999343


In [9]:
# entry_channel: Canal de captación del cliente

cca['entry_channel'].nunique() # 68
cca['entry_channel'].isnull().sum() # 133033
cca['entry_channel'].value_counts(dropna=False)

,count
entry_channel,
KHE,3113947
KFC,890620
KHQ,590280
KAT,416084
KHK,230197
...,...
KEJ,8
KHS,5
KDA,2


In [10]:
# Imputamos nulos con la moda
cca['entry_channel'] = cca['entry_channel'].fillna(cca['entry_channel'].mode()[0])

In [11]:
# active_customer: Indicador de actividad del cliente en la app EasyMoney

cca['active_customer'].nunique() # 2
cca['active_customer'].isnull().sum() # 0
cca['active_customer'].value_counts(dropna=False)

,count
active_customer,
0.0,3557573
1.0,2405351


In [12]:
# --- segment ---

cca['segment'].nunique() # 3
cca['segment'].isnull().sum() # 133944
pd.merge(cca['segment'].value_counts(dropna=False, normalize=True), cca['segment'].value_counts(dropna=False), left_index=True, right_index=True)

,proportion,count
segment,,
03 - UNIVERSITARIO,0.654069,3900166
02 - PARTICULARES,0.307043,1830875
NaN,0.022463,133944
01 - TOP,0.016425,97939


In [13]:
# Imputamos por moda tambien
cca['segment'] = cca['segment'].fillna(cca['segment'].mode()[0])

In [14]:
# la fila 448 de cca tiene nulos

cca[cca.isnull().any(axis=1)]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,new_customer


In [15]:
# eliminar la fila con los 4 nulos

cca = cca.dropna()

In [16]:
cca

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,new_customer
0,1375586,2018-01-01,2018-01-01,KHL,1.0,02 - PARTICULARES,0
1,1050611,2018-01-01,2015-08-01,KHE,0.0,03 - UNIVERSITARIO,0
2,1050612,2018-01-01,2015-08-01,KHE,0.0,03 - UNIVERSITARIO,0
3,1050613,2018-01-01,2015-08-01,KHD,0.0,03 - UNIVERSITARIO,0
4,1050614,2018-01-01,2015-08-01,KHE,1.0,03 - UNIVERSITARIO,0
...,...,...,...,...,...,...,...
13647304,1166765,2019-05-01,2016-08-01,KHE,0.0,03 - UNIVERSITARIO,0
13647305,1166764,2019-05-01,2016-08-01,KHE,0.0,03 - UNIVERSITARIO,0
13647306,1166763,2019-05-01,2016-08-01,KHE,1.0,02 - PARTICULARES,0
13647307,1166789,2019-05-01,2016-08-01,KHE,0.0,03 - UNIVERSITARIO,0


## DATASET 'COSTUMER_PRODUCTS.CSV'

In [17]:
df_customer_products = pd.read_csv(os.path.join(data_path, 'customer_products.csv'), index_col = 0)
df_customer_products

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13647304,1166765,2019-05,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
13647305,1166764,2019-05,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
13647306,1166763,2019-05,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
13647307,1166789,2019-05,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [18]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
dtypes: float64(2), int64(14), object(1)
memory usage: 818.9+ MB


In [19]:
# Convertir 'pk_partition' a un formato datetime

df_customer_products['pk_partition'] = pd.to_datetime(df_customer_products['pk_partition'], format='%Y-%m')

# Pandas muestra el día 01 de cada mes por defecto, pero no supone una complicación en ningún sentido.

In [20]:
df_customer_products.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [21]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             float64       
 11  pension_plan        float64       
 12  payroll_account     int64         
 13  emc_account         int64         
 14  debit_card          int64         
 15  em_account_p        int64         
 16  em_acount           int64         
dtypes: datetime64[ns](1), float64(2), int64(14)
memory usage: 818.9 MB


In [22]:
df_customer_products.describe()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
count,5.962924e+06,5962924,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5962924.0,5.962924e+06,5.962863e+06,5.962863e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06
mean,1.234930e+06,2018-10-03 16:23:50.368327680,2.581619e-03,7.848498e-05,5.433576e-05,3.370662e-03,3.711770e-03,1.688098e-02,0.0,1.188444e-02,3.451144e-02,3.652641e-02,5.524471e-02,5.483233e-02,9.441140e-02,5.701901e-06,7.348076e-01
min,1.589100e+04,2018-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.112532e+06,2018-07-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.231097e+06,2018-11-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.352339e+06,2019-02-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.553689e+06,2019-05-01 00:00:00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,1.623020e+05,NaN,5.074401e-02,8.858828e-03,7.371080e-03,5.795948e-02,6.081113e-02,1.288255e-01,0.0,1.083660e-01,1.825388e-01,1.875960e-01,2.284573e-01,2.276527e-01,2.924002e-01,2.387859e-03,4.414356e-01


In [23]:
# pk_partition (existe en 3 de los los datasets del tfm), muestra una actualización mensual
# a partir de la cual se puede ver en las demás columnas si los clientes han estado activos o no.

In [24]:
# Ejemplo con cliente '1375586'.

df_customer_products[df_customer_products['pk_cid'] == 1375586]

# No queda claro si la descripcion de las variables es correcta, porque pone que este cliente tuvo 'tenencia de tarjeta de débito'
# durante unos pocos meses, pero otros no. Se refiere a uso y no tenencia?

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1047196,1375586,2018-02-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1681276,1375586,2018-03-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2299297,1375586,2018-04-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2714121,1375586,2018-05-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0,1
3343336,1375586,2018-06-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0,1
4581564,1375586,2018-07-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0,1
5075380,1375586,2018-08-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
6258167,1375586,2018-09-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
6572487,1375586,2018-10-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [25]:
df_customer_products.duplicated().sum()

np.int64(0)

In [26]:
df_customer_products.isnull().sum()

,0
pk_cid,0
pk_partition,0
short_term_deposit,0
loans,0
mortgage,0
funds,0
securities,0
long_term_deposit,0
em_account_pp,0
credit_card,0


In [27]:
# Comprobación individual de columnas con nulos

for col in df_customer_products.columns:
  if df_customer_products[col].isnull().sum() > 0:
    print(df_customer_products[col].value_counts(dropna=False))
    print()
    print(df_customer_products[col].value_counts(normalize=True, dropna=False) * 100) # visualizacion en porcentajes
    print()

payroll
0.0    5757076
1.0     205787
NaN         61
Name: count, dtype: int64

payroll
0.0    96.547868
1.0     3.451109
NaN     0.001023
Name: proportion, dtype: float64

pension_plan
0.0    5745061
1.0     217802
NaN         61
Name: count, dtype: int64

pension_plan
0.0    96.346373
1.0     3.652604
NaN     0.001023
Name: proportion, dtype: float64



In [28]:
# Imputación de nulos por la moda

for col in df_customer_products.columns:

  if df_customer_products[col].isnull().sum() > 0:

    valor_moda = df_customer_products[col].mode()[0]
    df_customer_products[col].fillna(valor_moda, inplace=True)

/tmp/ipython-input-4020819241.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customer_products[col].fillna(valor_moda, inplace=True)


In [29]:
df_customer_products[['payroll', 'pension_plan']].isnull().sum()

,0
payroll,0
pension_plan,0


In [30]:
# Visualizacion como integers para que dataset sea mas uniforme

for col in df_customer_products.columns:
  if df_customer_products[col].dtype == 'float64':
    df_customer_products[col] = df_customer_products[col].astype('int64')

In [31]:
df_customer_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             int64         
 11  pension_plan        int64         
 12  payroll_account     int64         
 13  emc_account         int64         
 14  debit_card          int64         
 15  em_account_p        int64         
 16  em_acount           int64         
dtypes: datetime64[ns](1), int64(16)
memory usage: 818.9 MB


### Análisis univariable

In [32]:
for col in df_customer_products.columns:
    print(df_customer_products[col].value_counts())
    print()

pk_cid
538227     17
538736     17
539016     17
538902     17
538545     17
           ..
1005736     1
1008380     1
1022690     1
1090814     1
1090813     1
Name: count, Length: 456373, dtype: int64

pk_partition
2019-05-01    442995
2019-04-01    439627
2019-03-01    436183
2019-02-01    431727
2019-01-01    426875
2018-12-01    422481
2018-11-01    416387
2018-10-01    402300
2018-09-01    375323
2018-08-01    352922
2018-07-01    339339
2018-06-01    252104
2018-05-01    249926
2018-04-01    247463
2018-03-01    245258
2018-02-01    242521
2018-01-01    239493
Name: count, dtype: int64

short_term_deposit
0    5947530
1      15394
Name: count, dtype: int64

loans
0    5962456
1        468
Name: count, dtype: int64

mortgage
0    5962600
1        324
Name: count, dtype: int64

funds
0    5942825
1      20099
Name: count, dtype: int64

securities
0    5940791
1      22133
Name: count, dtype: int64

long_term_deposit
0    5862264
1     100660
Name: count, dtype: int64

em_account_p

## DATASET 'CUSTOMER_SOCIODEMOGRAPHICS.CSV'

In [33]:
sociodemographics = pd.read_csv(os.path.join(data_path, 'customer_sociodemographics.csv'),index_col=0)
sociodemographics

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN
...,...,...,...,...,...,...,...,...
13647304,1166765,2019-05,ES,50.0,V,22,N,43912.17
13647305,1166764,2019-05,ES,26.0,V,23,N,23334.99
13647306,1166763,2019-05,ES,50.0,H,47,N,NaN
13647307,1166789,2019-05,ES,50.0,H,22,N,199592.82


#### Tratamos cada una de las 8 columnas:

In [34]:
# pk_cid: Identificador de cliente

sociodemographics['pk_cid'].isnull().sum() # 0
sociodemographics['pk_cid'].nunique() # 456373 clientes diferentes

456373

In [35]:
# pk_partition: Identificador de partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes

# Pasamos a tipo de dato datetime
sociodemographics['pk_partition'] = pd.to_datetime(sociodemographics['pk_partition'])
sociodemographics['pk_partition'].isnull().sum() # 0

np.int64(0)

In [36]:
# deceased: Indicador de fallecimiento

# 0.000018% fallecidos, los eliminamos para simplificar el dataset y nos quitamos la columna de encima
sociodemographics['deceased'].value_counts(dropna=False, normalize=True)
sociodemographics = sociodemographics[sociodemographics['deceased'] == 'N'].drop(columns = 'deceased', axis=0)

In [37]:
# gender: Sexo del cliente

# Imputamos los nulos con la moda
sociodemographics['gender'].value_counts(dropna=False)  # 25 nulos
sociodemographics['gender'] = sociodemographics['gender'].fillna(sociodemographics['gender'].mode()[0])

In [38]:
# region_code: Provincia de residencia del cliente (para ES)

sociodemographics['region_code'].nunique() # 52
sociodemographics['region_code'].isnull().sum() # 2264: clientes que no viven en España.
sociodemographics[sociodemographics['country_id'] == 'ES']['region_code'].isnull().sum()
# No hay nulos que tengan countryID en españa


np.int64(12)

In [39]:
sociodemographics['region_code'].nunique()

52

In [40]:
# Imputamos estos nulos fuera de españa con el valor 00
sociodemographics['region_code'] = sociodemographics['region_code'].fillna(0) # Los imputamos con un nuevo valor 00 (fuera de ESpaña)
sociodemographics['region_code'].isnull().sum() # 0

np.int64(0)

In [41]:
# salary: Ingresos brutos de la unidad familiar

sociodemographics['salary'].isnull().sum() # 1541104

np.int64(1540876)

In [42]:
# Vemos que los menores de 18 tienen salario NaN, los imputamos con 0
sociodemographics[sociodemographics['age']<19].groupby(['age'])['salary'].mean()

sociodemographics.loc[sociodemographics['age'] < 18, 'salary'] = 0
sociodemographics['salary'].isnull().sum()

np.int64(1505003)

In [43]:
# El resto de nulos los imputamos con la media de cada grupo de region_code y edad
salario_sin_nulos = sociodemographics.groupby(['region_code', 'age'])['salary'].transform(lambda x: x.fillna(x.mean()))
salario_sin_nulos.isnull().sum() # 69858 nulos

# Como quedan nulos, los imputamos con la media global
salario_sin_nulos = salario_sin_nulos.fillna(salario_sin_nulos.mean())
sociodemographics['salary'] = salario_sin_nulos
sociodemographics['salary'].isnull().sum()  # 0

np.int64(0)

In [44]:
# country_id: Pais de residencia del cliente

sociodemographics['country_id'].isnull().sum() # 0
sociodemographics['country_id'].nunique() # 41
sociodemographics['country_id'].value_counts(dropna=False, normalize=True).head(5)

,proportion
country_id,
ES,0.999623
GB,0.000074
FR,0.000038
DE,0.000033
US,0.000033


In [45]:
# Eliminamos los que son fuera de España ya que representan menos de un 0.1% y nos quitamos la columna de encima

sociodemographics = sociodemographics[sociodemographics['country_id'] == 'ES']
sociodemographics = sociodemographics.drop(columns = 'country_id', axis=1)

In [46]:
# age: Edad del cliente

sociodemographics['age'].isnull().sum() # 0
sociodemographics['age'].value_counts(dropna=False).sort_index()

,count
age,
2,688
3,1520
4,2207
5,2920
6,3043
...,...
101,75
102,55
103,25


## DATASET 'SALES.CSV'

In [47]:
sales = pd.read_csv(os.path.join(data_path, 'sales.csv'),index_col=0)
sales

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05-01,2335,952.9
1,6667,35063,2018-06-01,2335,1625.2
2,6668,37299,2018-02-01,2335,1279.7
3,6669,39997,2018-02-01,2335,1511.9
4,6670,44012,2018-02-01,2335,1680.3
...,...,...,...,...,...
240768,247434,1553456,2019-05-01,4657,56.7
240769,247435,1553541,2019-05-01,4657,66.5
240770,247436,1553559,2019-05-01,4657,73.0
240771,247437,1553565,2019-05-01,4657,82.3


In [48]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   pk_sale     240773 non-null  int64  
 1   cid         240773 non-null  int64  
 2   month_sale  240773 non-null  object 
 3   product_ID  240773 non-null  int64  
 4   net_margin  240773 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 11.0+ MB


#### Tratamos cada una de las 5 columnas:

In [49]:
# pk_sale: Identificador de venta

sales['pk_sale'].nunique()
sales['pk_sale'].isnull().sum() # 0

np.int64(0)

In [50]:
# cid: Identificador de cliente

sales['cid'].nunique()
sales['cid'].isnull().sum() # 0

np.int64(0)

In [51]:
# month_sale: Mes de venta. Mes en el que se contrata un producto

# Pasamos a datetime
sales['month_sale'] = pd.to_datetime(sales['month_sale'])
sales['month_sale'].isnull().sum() # 0

np.int64(0)

In [52]:
# product_ID: Identificador de producto

sales['product_ID'].nunique() # 13
sales['product_ID'].isnull().sum() # 0
sales['product_ID'].value_counts(dropna=False)

,count
product_ID,
4657,117435
3819,38840
1364,19369
2234,18750
2235,18110
8871,12323
2312,5206
2335,4441
2336,4227


In [53]:
# net_margin: Margen neto generado por la venta ($)

sales['net_margin'].isnull().sum() # 0
sales['net_margin'].describe() # Detectamos un outlayer de 18466.4

,net_margin
count,240773.000000
mean,606.878164
std,1735.806089
min,25.000000
25%,61.400000
50%,69.100000
75%,79.200000
max,18466.400000


In [54]:
# A que products_ID diferentes pertenecen los sales con un 'net_margin' superior a 10000

sales[sales['net_margin'] > 10000]['product_ID'].value_counts()
# Vemos que el producto 1364 es el mas caro con lo que no es un outlier directamente

,count
product_ID,
1364,1043


## DATASET 'PRODUCT_DESCRIPTION.CSV'

In [55]:
product_description = pd.read_csv(os.path.join(data_path, 'product_description.csv'),index_col=0)
product_description

,pk_product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
6,2312,credit_card,payment_card
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment


## Datasets limpios

In [56]:
# Pasar a csv todos los dataframes limpios
# cca.to_csv(os.path.join(data_path, 'clean_datasets/customer_commercial_activity_clean.csv'))
# df_customer_products.to_csv(os.path.join(data_path, 'clean_datasets/customer_products_clean.csv'))
# sociodemographics.to_csv(os.path.join(data_path, 'clean_datasets/customer_sociodemographics_clean.csv'))
# sales.to_csv(os.path.join(data_path, 'clean_datasets/sales_clean.csv'))
# product_description.to_csv(os.path.join(data_path, 'clean_datasets/product_description_clean.csv'))